In [15]:
# Importación de librerías
import sqlite3
import pandas as pd

# Conectar a una base de datos SQLite en memoria (temporal para este ejemplo)
conn = sqlite3.connect(":memory:")

# Cargar el dataset de ciudades
url_cities = "https://raw.githubusercontent.com/datasets/world-cities/master/data/world-cities.csv"
cities = pd.read_csv(url_cities)
cities.to_sql('cities', conn, index=False, if_exists='replace')

# Cargar un dataset de empleados
url_employees = "https://raw.githubusercontent.com/justmarkham/DAT8/master/data/u.user"
employees = pd.read_csv(url_employees, sep="|")
employees.to_sql('employees', conn, index=False, if_exists='replace')

# Dataset de países (para relacionarlo con cities)
countries_data = {
    'country': ['Andorra', 'United Arab Emirates', 'United States', 'Canada'],
    'region': ['Europe', 'Middle East', 'North America', 'North America'],
    'population_millions': [0.077, 9.77, 331, 37.59]
}
countries = pd.DataFrame(countries_data)
countries.to_sql('countries', conn, index=False, if_exists='replace')

# Dataset de departamentos (para relacionarlo con employees)
departments_data = {
    'occupation': ['technician', 'writer', 'other', 'manager'],
    'department_name': ['Technical Support', 'Editorial', 'Miscellaneous', 'Management'],
    'budget_millions': [5.5, 3.0, 2.5, 10.0]
}
departments = pd.DataFrame(departments_data)
departments.to_sql('departments', conn, index=False, if_exists='replace')

In [16]:
## Ejercicio A: Contar Ciudades por País

print("\nEjercicio A - Contar Ciudades por País:")
qa1 = """
SELECT country, COUNT(name) AS city_count
FROM cities
GROUP BY country
HAVING COUNT(name) > 2;
"""
ra1 = pd.read_sql_query(qa1, conn)
print(ra1)

## Ejercicio A: Listado de Empleados y Departamentos
print("\nEjercicio A - Listado de Empleados y Departamentos:")
qa2 = """
SELECT e.user_id, e.age, e.gender, e.occupation, d.department_name
FROM employees e
JOIN departments d ON e.occupation = d.occupation;
"""
ra2 = pd.read_sql_query(qa2, conn)
print(ra2)

## Ejercicio A: Promedio de Edad por Ocupación
print("\nEjercicio A - Promedio de Edad por Ocupación:")
qa3 = """
SELECT occupation, AVG(age) AS average_age
FROM employees
GROUP BY occupation;
"""
ra3 = pd.read_sql_query(qa3, conn)
print(ra3)

## Ejercicio A: Ciudades y Región
print("\nEjercicio A - Ciudades y Región:")
qa4 = """
SELECT c.name AS city_name, c.country, co.region
FROM cities c
JOIN countries co ON c.country = co.country;
"""
ra4 = pd.read_sql_query(qa4, conn)
print(ra4)




## Ejercicio B: Calcular el Número de Ciudades por Región con CTE
print("\nEjercicio B - Calcular el Número de Ciudades por Región con CTE:")
qb1 = """
WITH CityCount AS (
    SELECT co.region, COUNT(c.name) AS city_count
    FROM cities c
    JOIN countries co ON c.country = co.country
    GROUP BY co.region
)
SELECT region, city_count
FROM CityCount
WHERE city_count > 3;
"""
rb1 = pd.read_sql_query(qb1, conn)
print(rb1)

## Ejercicio B: Ranking de Edad de Empleados por Ocupación con Window Functions
print("\nEjercicio B - Ranking de Edad de Empleados por Ocupación con Window Functions:")
qb2 = """
SELECT user_id, occupation, age,
       RANK() OVER (PARTITION BY occupation ORDER BY age DESC) AS age_rank
FROM employees
"""
rb2 = pd.read_sql_query(qb2, conn)
rb2 = rb2[rb2['age_rank'] <= 2]
print(rb2)

## Ejercicio B: Comparación de Ciudades y Población por Región
print("\nEjercicio B - Comparación de Ciudades y Población por Región:")
qb3 = """
SELECT co.region, COUNT(c.name) AS city_count, SUM(co.population_millions) AS total_population
FROM cities c
JOIN countries co ON c.country = co.country
GROUP BY co.region
HAVING COUNT(c.name) > 3 AND SUM(co.population_millions) > 50;
"""
rb3 = pd.read_sql_query(qb3, conn)
print(rb3)

## Ejercicio B: Empleados con Edad Superior al Promedio General en su Departamento
print("\nEjercicio B - Empleados con Edad Superior al Promedio General en su Departamento:")
qb4 = """
SELECT e.user_id, e.age, e.occupation
FROM employees e
WHERE e.age > (
    SELECT AVG(e2.age)
    FROM employees e2
    WHERE e2.occupation = e.occupation
);
"""
rb4 = pd.read_sql_query(qb4, conn)
print(rb4)




## Ejercicio C: Encontrar los Países con el Mayor Número de Ciudades y su Región
print("\nEjercicio C - Encontrar los Países con el Mayor Número de Ciudades y su Región:")
qc1 = """
WITH CityCountPerCountry AS (
    SELECT co.country, co.region, COUNT(c.name) AS city_count
    FROM cities c
    JOIN countries co ON c.country = co.country
    GROUP BY co.country, co.region
),
MaxCityCountPerRegion AS (
    SELECT region, MAX(city_count) AS max_city_count
    FROM CityCountPerCountry
    GROUP BY region
)
SELECT cpc.country, cpc.region, cpc.city_count
FROM CityCountPerCountry cpc
JOIN MaxCityCountPerRegion mcc ON cpc.region = mcc.region AND cpc.city_count = mcc.max_city_count;
"""
rc1 = pd.read_sql_query(qc1, conn)
print(rc1)

## Ejercicio C: Comparación de Presupuestos y Cantidad de Empleados por Departamento
print("\nEjercicio C - Comparación de Presupuestos y Cantidad de Empleados por Departamento:")
qc2 = """
WITH EmployeeCount AS (
    SELECT e.occupation, COUNT(e.user_id) AS employee_count
    FROM employees e
    GROUP BY e.occupation
)
SELECT d.department_name, d.budget_millions, ec.employee_count,
       (d.budget_millions / ec.employee_count) AS budget_per_employee
FROM departments d
JOIN EmployeeCount ec ON d.occupation = ec.occupation
WHERE (d.budget_millions / ec.employee_count) < 0.1;
"""
rc2 = pd.read_sql_query(qc2, conn)
print(rc2)



Ejercicio A - Contar Ciudades por País:
            country  city_count
0       Afghanistan          50
1           Albania          22
2           Algeria         292
3            Angola          95
4         Argentina         233
..              ...         ...
179        Viet Nam         118
180  Western Sahara           3
181           Yemen          23
182          Zambia          30
183        Zimbabwe          28

[184 rows x 2 columns]

Ejercicio A - Listado de Empleados y Departamentos:
     user_id  age gender  occupation    department_name
0          1   24      M  technician  Technical Support
1          2   53      F       other      Miscellaneous
2          3   23      M      writer          Editorial
3          4   24      M  technician  Technical Support
4          5   33      F       other      Miscellaneous
..       ...  ...    ...         ...                ...
172      914   44      F       other      Miscellaneous
173      919   25      M       other      Miscella